In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 21 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (15.0 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 124926 files and direc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import shap
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] =False

## Data Load

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/LG Aimers 6기/Random Forest/Data/train_2.csv')

In [ ]:
df = df.drop(columns = ['ID'])

In [ ]:
df.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256351 entries, 0 to 256350
Data columns (total 116 columns):
 #    Column                 Dtype  
---   ------                 -----  
 0    임신 시도 또는 마지막 임신 경과 연수  float64
 1    배란 자극 여부               bool   
 2    단일 배아 이식 여부            bool   
 3    착상 전 유전 검사 사용 여부       bool   
 4    착상 전 유전 진단 사용 여부       bool   
 5    남성 주 불임 원인             bool   
 6    남성 부 불임 원인             bool   
 7    여성 주 불임 원인             bool   
 8    여성 부 불임 원인             bool   
 9    부부 주 불임 원인             bool   
 10   부부 부 불임 원인             bool   
 11   불명확 불임 원인              bool   
 12   불임 원인 - 난관 질환          bool   
 13   불임 원인 - 남성 요인          bool   
 14   불임 원인 - 배란 장애          bool   
 15   불임 원인 - 여성 요인          bool   
 16   불임 원인 - 자궁경부 문제        bool   
 17   불임 원인 - 자궁내막증          bool   
 18   불임 원인 - 정자 농도          bool   
 19   불임 원인 - 정자 면역학적 요인     bool   
 20   불임 원인 - 정자 운동성         bool   
 21   불임 원인 - 정자 형태          bool   


In [ ]:
null_cols = ['시술 당시 나이_6', '배란 유도 유형_0', '난자 기증자 나이_6', '정자 기증자 나이_6', '난자 출처_2', '정자 출처_1', 'Unknown', '알 수 없음']

In [ ]:
df = df.drop(columns = null_cols)

### Library load

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 3.3 MB/s eta 0:00:00


In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.2 MB/s eta 0:00:00


In [ ]:
!pip install lightgbm

In [ ]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 3.4 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from skopt import BayesSearchCV
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


#### Optuna

##### prev

In [ ]:
# 데이터 분리
X = df.drop(columns=['임신 성공 여부'])
y = df['임신 성공 여부']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optuna 최적화 함수 정의
def objective(trial):
    # 최적화할 하이퍼파라미터 정의
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1, sum(y == 0) / sum(y == 1)),  # 클래스 불균형 조정
        "random_state": 42
    }

    # 모델 학습
    xgb_clf = xgb.XGBClassifier(**params, use_label_encoder=False, eval_metric='logloss')
    xgb_clf.fit(X_train, y_train)

    # 예측
    y_pred = xgb_clf.predict(X_test)
    y_pred_proba = xgb_clf.predict_proba(X_test)[:, 1]

    # 평가 지표 계산 (ROC-AUC 사용)
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    return roc_auc  # Optuna는 최대화할 값이므로 ROC-AUC 사용

# Optuna 최적화 실행
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)  # 50번 반복해서 최적의 하이퍼파라미터 찾기

# 최적의 하이퍼파라미터 출력
best_params = study.best_params
print("Best Hyperparameters:", best_params)

# 최적의 하이퍼파라미터로 모델 재학습
best_model = xgb.XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss')
best_model.fit(X_train, y_train)

# 최종 평가
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("\nFinal Model Performance:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")


[I 2025-02-22 07:54:04,629] A new study created in memory with name: no-name-ed3020d5-7359-41a8-99a8-0b4220d9330a
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [07:54:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2025-02-22 07:57:08,941] Trial 0 finished with value: 0.6973437343934517 and parameters: {'n_estimators': 842, 'max_depth': 14, 'learning_rate': 0.05368513577038917, 'subsample': 0.7420319927138395, 'colsample_bytree': 0.9170271719688204, 'scale_pos_weight': 2.4741859745868764}. Best is trial 0 with value: 0.6973437343934517.
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [07:57:10] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2025-02-22 07:57:48,183] Trial 1 finished with value: 0.7328024473045036 and parameters: {'n_estimators': 561, 'max_depth': 5, '

Best Hyperparameters: {'n_estimators': 613, 'max_depth': 6, 'learning_rate': 0.012935148894847554, 'subsample': 0.6246271083692173, 'colsample_bytree': 0.624479374048085, 'scale_pos_weight': 2.030169192399169}


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:29:55] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Final Model Performance:
Accuracy: 0.6921
Precision: 0.4257
Recall: 0.5798
F1-score: 0.4909
ROC-AUC: 0.7389


In [ ]:
X = df.drop(columns=['임신 성공 여부'])
y = df['임신 성공 여부']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

params = {
    "n_estimators": 613,
    "max_depth": 8,
    "learning_rate": 0.024410232383115377,
    "subsample": 0.5722176266655941,
    "colsample_bytree": 0.6120528781917591,
    "scale_pos_weight": 2.134890384642284,  # 클래스 불균형 조정
    "random_state": 42
}

base_models = [
    ('rf', RandomForestClassifier(n_estimators=700, max_depth=14, class_weight = 'balanced', random_state=42)),
    ('xgb', XGBClassifier(**params, use_label_encoder=False, eval_metric='logloss')),
    #('lgbm', LGBMClassifier(n_estimators=200, learning_rate=0.05, random_state=42))
]

final_estimator = XGBClassifier(n_estimators=200, learning_rate=0.05, random_state=42)
stacking_model = StackingClassifier(estimators=base_models,
                                    final_estimator=final_estimator)
stacking_model.fit(X_train, y_train)

y_pred = stacking_model.predict(X_test)
y_pred_proba = stacking_model.predict_proba(X_test)[:, 1]  # ROC-AUC

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")
print('\n')

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:37:21] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:52:24] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:53:10] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:53:55] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:54:39] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

Accuracy: 0.7469
Precision: 0.5257
Recall: 0.1162
F1-score: 0.1903
ROC-AUC: 0.7372




In [ ]:
X = df.drop(columns=['임신 성공 여부'])
y = df['임신 성공 여부']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 30),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        "class_weight": trial.suggest_categorical("class_weight", ["balanced", "balanced_subsample"]),
        "random_state": 42,
        "n_jobs": -1
    }

    rf_clf = RandomForestClassifier(**params)
    rf_clf.fit(X_train, y_train)

    y_pred = rf_clf.predict(X_test)
    y_pred_proba = rf_clf.predict_proba(X_test)[:, 1]

    roc_auc = roc_auc_score(y_test, y_pred_proba)

    return roc_auc

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

best_params = study.best_params
print("Best Hyperparameters:", best_params)

best_model = RandomForestClassifier(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("\nFinal Model Performance:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

[I 2025-02-22 13:13:43,127] A new study created in memory with name: no-name-7f1944b0-0b59-446b-86e4-1f90a863ef8b
[I 2025-02-22 13:16:02,722] Trial 0 finished with value: 0.7352536504181508 and parameters: {'n_estimators': 405, 'max_depth': 20, 'min_samples_split': 20, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'class_weight': 'balanced_subsample'}. Best is trial 0 with value: 0.7352536504181508.
[I 2025-02-22 13:36:52,048] Trial 1 finished with value: 0.7340793758996438 and parameters: {'n_estimators': 962, 'max_depth': 9, 'min_samples_split': 18, 'min_samples_leaf': 4, 'max_features': None, 'class_weight': 'balanced_subsample'}. Best is trial 0 with value: 0.7352536504181508.
[I 2025-02-22 13:38:08,420] Trial 2 finished with value: 0.7334504174598451 and parameters: {'n_estimators': 293, 'max_depth': 20, 'min_samples_split': 13, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.7352536504181508.
[I 2025-02-22 13:39:42,344] T

Best Hyperparameters: {'n_estimators': 915, 'max_depth': 23, 'min_samples_split': 11, 'min_samples_leaf': 10, 'max_features': 'sqrt', 'class_weight': 'balanced'}

Final Model Performance:
Accuracy: 0.6361
Precision: 0.3890
Recall: 0.7386
F1-score: 0.5096
ROC-AUC: 0.7358


In [ ]:
import optuna
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 데이터 분리
X = df.drop(columns=['임신 성공 여부'])
y = df['임신 성공 여부']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optuna 최적화 함수 정의
def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "depth": trial.suggest_int("depth", 4, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 1, 10),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "loss_function": "Logloss",
        "eval_metric": "AUC",
        "random_seed": 42,
        "verbose": 0,
        #"task_type": "CPU"  # CPU 사용
    }

    model = cb.CatBoostClassifier(**params)
    model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=0)

    y_pred_proba = model.predict_proba(X_test)[:, 1]
    return roc_auc_score(y_test, y_pred_proba)

# Optuna 최적화 실행
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# 최적의 하이퍼파라미터 출력
best_params = study.best_params
print("Best Hyperparameters:", best_params)

# 최적의 하이퍼파라미터로 모델 재학습
best_model = cb.CatBoostClassifier(**best_params)#, task_type="CPU")
best_model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=100)

# 최종 평가
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("\nFinal Model Performance:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")


[I 2025-02-22 18:51:57,552] A new study created in memory with name: no-name-638a51d5-de55-43c5-acab-5a558237b514
[I 2025-02-22 18:52:24,786] Trial 0 finished with value: 0.7384212211249459 and parameters: {'iterations': 649, 'depth': 5, 'learning_rate': 0.13620030828837193, 'l2_leaf_reg': 9, 'border_count': 214, 'bagging_temperature': 0.3335677783778248}. Best is trial 0 with value: 0.7384212211249459.
[I 2025-02-22 18:52:43,171] Trial 1 finished with value: 0.7384646890288301 and parameters: {'iterations': 399, 'depth': 8, 'learning_rate': 0.2417039622031578, 'l2_leaf_reg': 7, 'border_count': 230, 'bagging_temperature': 0.08598743717950363}. Best is trial 1 with value: 0.7384646890288301.
[I 2025-02-22 18:53:00,196] Trial 2 finished with value: 0.7374848866448181 and parameters: {'iterations': 453, 'depth': 9, 'learning_rate': 0.29543920996739315, 'l2_leaf_reg': 10, 'border_count': 144, 'bagging_temperature': 0.335721614773557}. Best is trial 1 with value: 0.7384646890288301.
[I 2025

Best Hyperparameters: {'iterations': 851, 'depth': 8, 'learning_rate': 0.04530226945582808, 'l2_leaf_reg': 4, 'border_count': 251, 'bagging_temperature': 0.3889498313962164}
0:	learn: 0.6634229	test: 0.6630541	best: 0.6630541 (0)	total: 62.5ms	remaining: 53.1s
100:	learn: 0.4880365	test: 0.4884215	best: 0.4884215 (100)	total: 6.24s	remaining: 46.3s
200:	learn: 0.4839714	test: 0.4868316	best: 0.4868316 (200)	total: 13.6s	remaining: 44s
300:	learn: 0.4811102	test: 0.4863423	best: 0.4863423 (300)	total: 19.5s	remaining: 35.6s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.4863274012
bestIteration = 301

Shrink model to first 302 iterations.

Final Model Performance:
Accuracy: 0.7486
Precision: 0.5433
Recall: 0.1122
F1-score: 0.1860
ROC-AUC: 0.7386


In [ ]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 데이터 분리
X = df.drop(columns=['임신 성공 여부'])
y = df['임신 성공 여부']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optuna 최적화 함수 정의
def objective(trial):
    # 최적화할 하이퍼파라미터 정의
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", -1, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 300),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 50),
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1, sum(y == 0) / sum(y == 1)),  # 클래스 불균형 조정
        "random_state": 42,
        "n_jobs": -1,  # CPU 병렬 처리 활성화

        # 불필요한 메시지 숨기기
        "verbose": -1,
        "logging_level": "error"
    }

    # 모델 학습
    lgb_clf = lgb.LGBMClassifier(**params)
    lgb_clf.fit(X_train, y_train)

    # 예측
    y_pred_proba = lgb_clf.predict_proba(X_test)[:, 1]

    # 평가 지표 계산 (ROC-AUC 사용)
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    return roc_auc  # Optuna는 최대화할 값이므로 ROC-AUC 사용

# Optuna 최적화 실행
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)  # 50번 반복해서 최적의 하이퍼파라미터 찾기

# 최적의 하이퍼파라미터 출력
best_params = study.best_params
print("Best Hyperparameters:", best_params)

# 최적의 하이퍼파라미터로 모델 재학습
best_model = lgb.LGBMClassifier(**best_params)
best_model.fit(X_train, y_train)

# 최종 평가
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("\nFinal Model Performance:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")


[I 2025-02-22 19:22:49,330] A new study created in memory with name: no-name-dd38c965-0ff5-4d87-ab29-930b48f99619
[I 2025-02-22 19:23:01,132] Trial 0 finished with value: 0.7091964475434133 and parameters: {'n_estimators': 276, 'max_depth': 14, 'learning_rate': 0.2516489023677961, 'num_leaves': 250, 'subsample': 0.8205716872390754, 'colsample_bytree': 0.5472692198432654, 'min_child_samples': 27, 'scale_pos_weight': 2.2527565788373667}. Best is trial 0 with value: 0.7091964475434133.
[I 2025-02-22 19:23:27,178] Trial 1 finished with value: 0.7285116295317073 and parameters: {'n_estimators': 969, 'max_depth': 9, 'learning_rate': 0.060266118511671476, 'num_leaves': 148, 'subsample': 0.8930739517291548, 'colsample_bytree': 0.9974677306232757, 'min_child_samples': 49, 'scale_pos_weight': 2.1156734583107455}. Best is trial 1 with value: 0.7285116295317073.
[I 2025-02-22 19:23:35,214] Trial 2 finished with value: 0.7100055269587584 and parameters: {'n_estimators': 220, 'max_depth': 15, 'learn

Best Hyperparameters: {'n_estimators': 782, 'max_depth': 6, 'learning_rate': 0.015860873943021367, 'num_leaves': 147, 'subsample': 0.8115282675859632, 'colsample_bytree': 0.775903861904495, 'min_child_samples': 39, 'scale_pos_weight': 1.4725395190171597}

Final Model Performance:
Accuracy: 0.7332
Precision: 0.4732
Recall: 0.3703
F1-score: 0.4155
ROC-AUC: 0.7391


In [ ]:
X = df.drop(columns=['임신 성공 여부'])
y = df['임신 성공 여부']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

params = {'n_estimators': 613,
 'max_depth': 6,
 'learning_rate': 0.012935148894847554,
 'subsample': 0.6246271083692173,
 'colsample_bytree': 0.624479374048085,
 'scale_pos_weight': 2.030169192399169
}

xgb_clf = XGBClassifier(**params, use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)

y_pred = xgb_clf.predict(X_test)
y_pred_proba = xgb_clf.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

#print(f"n_estimators={n_trees}, max_depth={depth}, class_weight={weight}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")
print('\n')

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:01:29] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.6921
Precision: 0.4257
Recall: 0.5798
F1-score: 0.4909
ROC-AUC: 0.7389




##### HistoGradientBoosting

In [ ]:
import optuna
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optuna 최적화 함수 정의
def objective(trial):
    params = {
        "max_iter": trial.suggest_int("max_iter", 50, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
        "l2_regularization": trial.suggest_float("l2_regularization", 0.0, 10.0),
        "max_bins": trial.suggest_int("max_bins", 32, 255),
        "random_state": 42
    }

    model = HistGradientBoostingClassifier(**params)
    model.fit(X_train, y_train)

    y_pred_proba = model.predict_proba(X_test)[:, 1]
    return roc_auc_score(y_test, y_pred_proba)

# Optuna 실행
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# 최적의 하이퍼파라미터 출력
best_params = study.best_params
print("Best Parameters (HGBT):", best_params)

# 최적의 모델 학습
best_model = HistGradientBoostingClassifier(**best_params)
best_model.fit(X_train, y_train)

# 최종 평가
y_pred_proba = best_model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"Final ROC-AUC (HGBT): {roc_auc:.4f}")


/usr/local/lib/python3.11/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:19: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(
[I 2025-02-26 12:08:30,529] A new study created in memory with name: no-name-60a5c836-6e95-4ec6-adee-c767bbe22776
[I 2025-02-26 12:09:07,236] Trial 0 finished with value: 0.7386242144785149 and parameters: {'max_iter': 918, 'max_depth': 15, 'learning_rate': 0.0676642399622053, 'min_samples_leaf': 20, 'l2_regularization': 8.41822986341261, 'max_bins': 252}. Best is trial 0 with value: 0.7386242144785149.
[I 2025-02-26 12:10:05,504] Trial 1 finished with value: 0.7387138575433919 and parameters: {'max_iter': 615, 'max_depth': 9, 'learning_rate': 0.032173653080908075, 'min_samples_leaf': 9, 'l2_regularization': 1.7737314996022513, 'max_bins': 81}. Be

##### H2o

In [ ]:
!pip install h2o

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.8/265.8 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.46.0.6-py2.py3-none-any.whl size=265859786 sha256=467d90f6006681751a2ced317ed7155f1d55e0b4c8e0de6f5e9c76631b8f0f10
  Stored in directory: /root/.cache/pip/wheels/62/f9/aa/687bd54342d2981bc78e22ee9b9bc39f92006e344e7aa1e0ac
Successfully built h2o


In [ ]:
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.gbm import H2OGradientBoostingEstimator
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import pandas as pd

# H2O 서버 시작
h2o.init()

X = df.drop(columns=['임신 성공 여부'])
y = df['임신 성공 여부']

# 데이터 변환
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train = h2o.H2OFrame(pd.concat([X_train, y_train], axis=1))
test = h2o.H2OFrame(pd.concat([X_test, y_test], axis=1))

# 타겟 변수 명시
target = '임신 성공 여부'
features = [col for col in train.columns if col != target]
train[target] = train[target].asfactor()
test[target] = test[target].asfactor()

# Optuna 최적화 함수 정의
def objective(trial):
    params = {
        "ntrees": trial.suggest_int("ntrees", 50, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learn_rate": trial.suggest_float("learn_rate", 0.001, 0.01, log=True),
        "sample_rate": trial.suggest_float("sample_rate", 0.5, 1.0),
        "col_sample_rate": trial.suggest_float("col_sample_rate", 0.5, 1.0),
        "seed": 42
    }

    model = H2OGradientBoostingEstimator(**params)
    model.train(x=features, y=target, training_frame=train)

    preds = model.predict(test).as_data_frame()['p1']
    return roc_auc_score(y_test, preds)

# Optuna 실행
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# 최적의 하이퍼파라미터 출력
best_params = study.best_params
print("Best Parameters (H2O GBM):", best_params)

# 최적의 모델 학습
best_model = H2OGradientBoostingEstimator(**best_params)
best_model.train(x=features, y=target, training_frame=train)

# 최종 평가
preds = best_model.predict(test).as_data_frame()['p1']
roc_auc = roc_auc_score(y_test, preds)
print(f"Final ROC-AUC (H2O GBM): {roc_auc:.4f}")

# H2O 서버 종료
h2o.shutdown()


Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,44 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,3 months and 24 days
H2O_cluster_name:,H2O_from_python_unknownUser_olngnx
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


[I 2025-02-26 12:23:23,778] A new study created in memory with name: no-name-fd64ddad-fea8-48bb-94db-a5d13fda6b31


gbm Model Build progress: |

/usr/local/lib/python3.11/dist-packages/h2o/estimators/estimator_base.py:192: RuntimeWarning: Dropping bad and constant columns: [착상 전 유전 검사 사용 여부, 불임 원인 - 여성 요인, 배란 유도 유형_1, 난자 채취 경과일]
  warnings.warn(mesg["message"], RuntimeWarning)


██████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


/usr/local/lib/python3.11/dist-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
[W 2025-02-26 12:41:15,157] Trial 0 failed with parameters: {'ntrees': 936, 'max_depth': 9, 'learn_rate': 0.0016410414458118564, 'sample_rate': 0.5940159574928136, 'col_sample_rate': 0.6642851238023568} because of the following error: KeyError('p1').
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngi

KeyError: 'p1'

In [ ]:
# 예측 실행 후 컬럼 확인
preds = model.predict(test).as_data_frame()
print(preds.head())  # 예측 결과의 컬럼명 확인

# p1이 없으면 'predict' 컬럼을 사용
if 'p1' in preds.columns:
    y_pred_proba = preds['p1']
else:
    y_pred_proba = preds['predict']  # H2O는 이진 분류에서 'predict'만 출력할 수도 있음

# ROC-AUC 계산
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"Final ROC-AUC (H2O GBM): {roc_auc:.4f}")


NameError: name 'model' is not defined

In [ ]:
!pip install ngboost

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 7.8 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=4480220ffc19dc6c094a78fdba04142f6c18f03b4d0346419572cadf920c8245
  Stored in directory: /root/.cache/pip/wheels/8b/67/f4/2caaae2146198dcb824f31a303833b07b14a5ec863fb3acd7b
Successfully built autograd-gamma


In [ ]:
import optuna
from ngboost import NGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from ngboost.distns import Bernoulli
from ngboost.scores import LogScore

X = df.drop(columns=['임신 성공 여부'])
y = df['임신 성공 여부']

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optuna 최적화 함수 정의
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        "minibatch_frac": trial.suggest_float("minibatch_frac", 0.5, 1.0),
        "natural_gradient": trial.suggest_categorical("natural_gradient", [True, False]),
        "random_state": 42
    }

    model = NGBClassifier(Dist=Bernoulli, Score=LogScore, **params)
    model.fit(X_train, y_train)

    y_pred_proba = model.predict_proba(X_test)[:, 1]
    return roc_auc_score(y_test, y_pred_proba)

# Optuna 실행
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# 최적의 하이퍼파라미터 출력
best_params = study.best_params
print("Best Parameters (NGBoost):", best_params)

# 최적의 모델 학습
best_model = NGBClassifier(Dist=Bernoulli, Score=LogScore, **best_params)
best_model.fit(X_train, y_train)

# 최종 평가
y_pred_proba = best_model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"Final ROC-AUC (NGBoost): {roc_auc:.4f}")


[I 2025-02-26 12:57:10,816] A new study created in memory with name: no-name-a0801a63-b5df-4929-9c73-fd65598c6625
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[W 2025-02-26 12:57:13,850] Trial 0 failed with parameters: {'n_estimators': 869, 'learning_rate': 0.005560465634239584, 'minibatch_frac': 0.7285045382807094, 'natural_gradient': False} because of the following error: IndexError('boolean index did not match indexed array along dimension 0; dimension is 2 but corresponding boolean dimension is 149401').
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "<ipython-input-16-d787006d78a8>", line 25, in objective
    model.fit(X_train, y_train)
  File "/usr/local/lib/python3.11/dis

IndexError: boolean index did not match indexed array along dimension 0; dimension is 2 but corresponding boolean dimension is 149401

##### test

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/LG Aimers 6기/Random Forest/Data/test.csv')

In [ ]:
test_df = test_df.drop(columns = ['ID'])

In [ ]:
X_test = test_df
y_pred = xgb_clf.predict(X_test)
y_pred_proba = xgb_clf.predict_proba(X_test)[:, 1]

In [ ]:
num_samples = len(y_pred_proba)
ids = [f"TEST_{i:05d}" for i in range(num_samples)]

results_df = pd.DataFrame({
    "ID": ids,
    "probability": y_pred_proba
})

results_df.head()

,ID,probability
0,TEST_00000,0.006760
1,TEST_00001,0.003234
2,TEST_00002,0.276104
3,TEST_00003,0.200099
4,TEST_00004,0.681818


In [ ]:
results_df.to_csv('/content/drive/MyDrive/LG Aimers 6기/Random Forest/Prediction/prediction_0222_1.csv', index=False, encoding='utf-8-sig')

#### Bayes

##### XGBoost

In [ ]:
X = df.drop(columns=['임신 성공 여부'])
y = df['임신 성공 여부']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

search_space = {
    'n_estimators': (100, 1000),
    'max_depth': (3, 15),
    'learning_rate': (0.001, 0.1, 'log-uniform'),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'scale_pos_weight': (0.5, 10)
}

bayes_search = BayesSearchCV(
    XGBClassifier(use_label_encoder=False, eval_metric='auc', random_state=42, verbosity=0),
    search_spaces=search_space,
    n_iter=30,  # 탐색 횟수 (클수록 성능이 좋아짐)
    scoring='roc_auc',
    cv=3,  # 3-Fold CV
    n_jobs=-1,
    random_state=42
)

bayes_search.fit(X_train, y_train)
print("Best Parameters:", bayes_search.best_params_)

best_model = bayes_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC-AUC: {roc_auc:.4f}")

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best Parameters: OrderedDict([('colsample_bytree', 0.7224162561505759), ('learning_rate', 0.06877728743793542), ('max_depth', 4), ('n_estimators', 490), ('scale_pos_weight', 2.2855742704898114), ('subsample', 0.7268326719031495)])
ROC-AUC: 0.7385


In [ ]:
X = df.drop(columns=['임신 성공 여부'])
y = df['임신 성공 여부']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

search_space = {
    'n_estimators': (100, 1000),
    'max_depth': (3, 15),
    'learning_rate': (0.001, 0.01, 'log-uniform'),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'scale_pos_weight': (0.5, 10)
}

bayes_search = BayesSearchCV(
    XGBClassifier(use_label_encoder=False, eval_metric='auc', random_state=42, verbosity=0),
    search_spaces=search_space,
    n_iter=20,  # 탐색 횟수 (클수록 성능이 좋아짐)
    scoring='roc_auc',
    cv=3,  # 3-Fold CV
    n_jobs=-1,
    random_state=42
)

bayes_search.fit(X_train, y_train)
print("Best Parameters:", bayes_search.best_params_)

best_model = bayes_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC-AUC: {roc_auc:.4f}")

##### LightGBM

In [ ]:
search_space_lgbm = {
    'n_estimators': (100, 1000),
    'max_depth': (3, 15),
    'learning_rate': (0.01, 0.3, 'log-uniform'),
    'num_leaves': (20, 300),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'scale_pos_weight': (0.5, 10)
}

bayes_search_lgbm = BayesSearchCV(
    LGBMClassifier(random_state=42, verbose=-1),
    search_spaces=search_space_lgbm,
    n_iter=50,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
    random_state=42
)

bayes_search_lgbm.fit(X_train, y_train)
print("Best Parameters (LightGBM):", bayes_search_lgbm.best_params_)

best_model = bayes_search_lgbm.best_estimator_
y_pred_proba = best_model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC-AUC: {roc_auc:.4f}")

Best Parameters (LightGBM): OrderedDict([('colsample_bytree', 1.0), ('learning_rate', 0.02119434750407919), ('max_depth', 4), ('n_estimators', 1000), ('num_leaves', 20), ('scale_pos_weight', 0.5), ('subsample', 1.0)])
ROC-AUC: 0.7386


In [ ]:
search_space_lgbm = {
    'n_estimators': (100, 1000),
    'max_depth': (3, 15),
    'learning_rate': (0.001, 0.1, 'log-uniform'),
    'num_leaves': (20, 300),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'scale_pos_weight': (0.5, 10),
    'min_child_samples': (10, 50),  # 너무 작은 값은 과적합 위험
    'reg_alpha': (0.0, 10.0),  # L1 정규화
    'reg_lambda': (0.0, 10.0),  # L2 정규화
}


bayes_search_lgbm = BayesSearchCV(
    LGBMClassifier(random_state=42, verbose=-1),
    search_spaces=search_space_lgbm,
    n_iter=30,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
    random_state=42
)

bayes_search_lgbm.fit(X_train, y_train)
print("Best Parameters (LightGBM):", bayes_search_lgbm.best_params_)

best_model = bayes_search_lgbm.best_estimator_
y_pred_proba = best_model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC-AUC: {roc_auc:.4f}")

Best Parameters (LightGBM): OrderedDict([('colsample_bytree', 0.5), ('learning_rate', 0.04580593375904727), ('max_depth', 3), ('min_child_samples', 43), ('n_estimators', 743), ('num_leaves', 233), ('reg_alpha', 2.3706590234907834), ('reg_lambda', 5.997149692748311), ('scale_pos_weight', 2.4884572620539993), ('subsample', 0.7760584176041248)])
ROC-AUC: 0.7389


In [ ]:
search_space_lgbm = {
    'n_estimators': (100, 1000),
    'max_depth': (3, 15),
    'learning_rate': (0.001, 0.1, 'log-uniform'),
    'num_leaves': (20, 300),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'scale_pos_weight': (0.5, 10),
    'min_child_samples': (10, 50),  # 너무 작은 값은 과적합 위험
    'reg_alpha': (0.0, 10.0),  # L1 정규화
    #'reg_lambda': (0.0, 10.0),  # L2 정규화
}


bayes_search_lgbm = BayesSearchCV(
    LGBMClassifier(random_state=42, verbose=-1),
    search_spaces=search_space_lgbm,
    n_iter=30,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
    random_state=42
)

bayes_search_lgbm.fit(X_train, y_train)
print("Best Parameters (LightGBM):", bayes_search_lgbm.best_params_)

best_model = bayes_search_lgbm.best_estimator_
y_pred_proba = best_model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC-AUC: {roc_auc:.4f}")

Best Parameters (LightGBM): OrderedDict([('colsample_bytree', 0.5), ('learning_rate', 0.1), ('max_depth', 3), ('min_child_samples', 39), ('n_estimators', 556), ('num_leaves', 20), ('reg_alpha', 10.0), ('scale_pos_weight', 1.6116066319368096), ('subsample', 0.5)])
ROC-AUC: 0.7393


In [ ]:
X = df.drop(columns=['임신 성공 여부', '임신 시도 또는 마지막 임신 경과 연수'])
y = df['임신 성공 여부']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

search_space_lgbm = {
    'n_estimators': (100, 1000),
    'max_depth': (3, 15),
    'learning_rate': (0.001, 0.1, 'log-uniform'),
    'num_leaves': (20, 300),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'scale_pos_weight': (0.5, 10),
    'min_child_samples': (10, 50),  # 너무 작은 값은 과적합 위험
    'reg_alpha': (0.0, 10.0),  # L1 정규화
    #'reg_lambda': (0.0, 10.0),  # L2 정규화
}


bayes_search_lgbm = BayesSearchCV(
    LGBMClassifier(random_state=42, verbose=-1),
    search_spaces=search_space_lgbm,
    n_iter=30,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
    random_state=42
)

bayes_search_lgbm.fit(X_train, y_train)
print("Best Parameters (LightGBM):", bayes_search_lgbm.best_params_)

best_model = bayes_search_lgbm.best_estimator_
y_pred_proba = best_model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC-AUC: {roc_auc:.4f}")

Best Parameters (LightGBM): OrderedDict([('colsample_bytree', 0.606854460834807), ('learning_rate', 0.08619071811281552), ('max_depth', 14), ('min_child_samples', 19), ('n_estimators', 122), ('num_leaves', 20), ('reg_alpha', 7.964111854915997), ('scale_pos_weight', 7.243210506519842), ('subsample', 0.7250303879006701)])
ROC-AUC: 0.7389


##### CatBoost

In [ ]:
search_space_catboost = {
    'iterations': (100, 1000),
    'depth': (4, 10),
    'learning_rate': (0.01, 0.3, 'log-uniform'),
    'l2_leaf_reg': (1, 10),
    'border_count': (32, 255),
    'bagging_temperature': (0.0, 1.0),
    'scale_pos_weight': (0.5, 10)
}

bayes_search_cat = BayesSearchCV(
    CatBoostClassifier(verbose=0, random_seed=42),
    search_spaces=search_space_catboost,
    n_iter=50,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
    random_state=42
)

bayes_search_cat.fit(X_train, y_train)
print("Best Parameters (CatBoost):", bayes_search_cat.best_params_)

best_model = bayes_search_cat.best_estimator_
y_pred_proba = best_model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC-AUC: {roc_auc:.4f}")

Best Parameters (CatBoost): OrderedDict([('bagging_temperature', 0.39593042882451235), ('border_count', 34), ('depth', 6), ('iterations', 674), ('l2_leaf_reg', 9), ('learning_rate', 0.02686602624485201), ('scale_pos_weight', 5.953042497685145)])
ROC-AUC: 0.7389


In [ ]:
search_space_catboost = {
    'iterations': (100, 1000),
    'depth': (4, 10),
    'learning_rate': (0.001, 0.01, 'log-uniform'),
    'l2_leaf_reg': (1, 10),
    'border_count': (32, 255),
    'bagging_temperature': (0.0, 1.0),
    'scale_pos_weight': (0.5, 10)
}

bayes_search_cat = BayesSearchCV(
    CatBoostClassifier(verbose=0, random_seed=42),
    search_spaces=search_space_catboost,
    n_iter=50,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
    random_state=42
)

bayes_search_cat.fit(X_train, y_train)
print("Best Parameters (CatBoost):", bayes_search_cat.best_params_)

best_model = bayes_search_cat.best_estimator_
y_pred_proba = best_model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC-AUC: {roc_auc:.4f}")

Best Parameters (CatBoost): OrderedDict([('bagging_temperature', 0.5263981365912495), ('border_count', 32), ('depth', 10), ('iterations', 892), ('l2_leaf_reg', 5), ('learning_rate', 0.01), ('scale_pos_weight', 1.5895976463808386)])
		ROC-AUC: 0.7390

Optuna

In [ ]:
import optuna
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 500, 2000),
        'depth': trial.suggest_int('depth', 4, 12),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.01),
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 100),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'random_strength': trial.suggest_float('random_strength', 0.0, 10.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 50),
        'grow_policy': trial.suggest_categorical('grow_policy', ['Depthwise', 'Lossguide']),
        'auto_class_weights': trial.suggest_categorical('auto_class_weights', ['Balanced', 'SqrtBalanced']),
        'loss_function': 'Logloss',
        'eval_metric': 'AUC',
        'random_seed': 42,
        'verbose': 0
    }

    model = CatBoostClassifier(**params)
    model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=0)

    y_pred_proba = model.predict_proba(X_test)[:, 1]
    return roc_auc_score(y_test, y_pred_proba)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print("Best Hyperparameters:", study.best_params)


[I 2025-02-25 19:06:50,501] A new study created in memory with name: no-name-9e536c39-32b8-4be8-be07-a0feaeb0ba50
<ipython-input-16-c4c68b8f449d>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.01),
[I 2025-02-25 19:12:28,567] Trial 0 finished with value: 0.7333577775382423 and parameters: {'iterations': 1026, 'depth': 11, 'learning_rate': 0.0030070642160528608, 'l2_leaf_reg': 85, 'bagging_temperature': 0.7490687007286356, 'border_count': 180, 'random_strength': 6.024028067428123, 'min_data_in_leaf': 26, 'grow_policy': 'Depthwise', 'auto_class_weights': 'Balanced'}. Best is trial 0 with value: 0.7333577775382423.
<ipython-input-16-c4c68b8f449d>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https

Best Hyperparameters: {'iterations': 1657, 'depth': 10, 'learning_rate': 0.009613534679162686, 'l2_leaf_reg': 82, 'bagging_temperature': 0.40999342662591226, 'border_count': 176, 'random_strength': 1.371428847017544, 'min_data_in_leaf': 38, 'grow_policy': 'Depthwise', 'auto_class_weights': 'SqrtBalanced'}


In [ ]:
X = df.drop(columns=['임신 성공 여부'])
y = df['임신 성공 여부']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

cat_params = {
    'iterations': 1657,
    'depth': 10,
    'learning_rate': 0.009613534679162686,
    'l2_leaf_reg': 82,
    'bagging_temperature': 0.40999342662591226,
    'border_count': 176,
    'random_strength': 1.371428847017544,
    'min_data_in_leaf': 38,
    'grow_policy': 'Depthwise',
    'auto_class_weights': 'SqrtBalanced'
}

cat_clf = CatBoostClassifier(**cat_params)
cat_clf.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=0)#early_stopping_rounds=50, verbose=0)

cat_pred = cat_clf.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, cat_pred)
print(f"ROC-AUC: {roc_auc:.6f}")

ROC-AUC: 0.739486


#### Blending

In [ ]:
rf_params = {'n_estimators': 550,
             'max_depth': 28,
             'min_samples_split': 4,
             'min_samples_leaf': 9,
             'max_features': 'sqrt',
             'class_weight': 'balanced_subsample'}

# xg_params = {
#     'colsample_bytree': 1.0,
#     'learning_rate': 0.01556331559557309,
#     'max_depth': 5,
#     'n_estimators': 794,
#     'scale_pos_weight': 0.5,
#     'subsample': 0.5,
#     "random_state": 42
# }

xg_params = {
    'colsample_bytree': 1.0,
    'learning_rate': 0.01,
    'max_depth': 6,
    'n_estimators': 1000,
    'scale_pos_weight': 0.5,
    'subsample': 0.5
}

# light_params = {
#     'n_estimators': 150,
#     'max_depth': 6,
#     'learning_rate': 0.02,
#     'num_leaves': 147,
#     'subsample': 0.81,
#     'colsample_bytree': 0.78,
#     'min_child_samples': 39,
#     'scale_pos_weight': 1.47
# }

# light_params = {
#     'colsample_bytree': 0.5,
#     'learning_rate': 0.01,
#     'max_depth': 7,
#     'n_estimators': 1000,
#     'num_leaves': 22,
#     'scale_pos_weight': 7.216461967272949,
#     'subsample': 0.6233977957706829
# }

# light_params = {
#     'n_estimators': 782,
#     'max_depth': 6,
#     'learning_rate': 0.015860873943021367,
#     'num_leaves': 147,
#     'subsample': 0.8115282675859632,
#     'colsample_bytree': 0.775903861904495,
#     'min_child_samples': 39,
#     'scale_pos_weight': 1.4725395190171597
# }

light_params = {
    'colsample_bytree': 0.5,
    'learning_rate': 0.1,
    'max_depth': 3,
    'min_child_samples': 39,
    'n_estimators': 556,
    'num_leaves': 20,
    'reg_alpha': 10.0,
    'scale_pos_weight': 1.6116066319368096,
    'subsample': 0.5
}

# cat_params = {
#     'iterations': 850,
#     'depth': 8,
#     'learning_rate': 0.05,
#     'l2_leaf_reg': 4,
#     'border_count': 251,
#     'bagging_temperature': 0.39
# }

# cat_params = {
#     'bagging_temperature': 1.0,
#     'border_count': 32,
#     'depth': 6,
#     'iterations': 727,
#     'l2_leaf_reg': 10,
#     'learning_rate': 0.03839813136390658,
#     'scale_pos_weight': 0.5
# }

cat_params = {
    'iterations': 1657,
    'depth': 10,
    'learning_rate': 0.009613534679162686,
    'l2_leaf_reg': 82,
    'bagging_temperature': 0.40999342662591226,
    'border_count': 176,
    'random_strength': 1.371428847017544,
    'min_data_in_leaf': 38,
    'grow_policy': 'Depthwise',
    'auto_class_weights': 'SqrtBalanced'
}

In [ ]:
X = df.drop(columns=['임신 성공 여부'])
y = df['임신 성공 여부']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

cat_clf = CatBoostClassifier(**cat_params)
cat_clf.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=0)#early_stopping_rounds=50, verbose=0)

lgb_clf = LGBMClassifier(**light_params, random_state = 42, verbose=-1)
lgb_clf.fit(X_train, y_train)

xgb_clf = XGBClassifier(**xg_params, use_label_encoder=False, eval_metric='auc', verbosity=0)
xgb_clf.fit(X_train, y_train)

# rf_clf = RandomForestClassifier(**rf_params)
# rf_clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
cat_clf_ = CatBoostClassifier(**cat_params)
cat_clf_.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=0)

In [ ]:
xgb_pred = xgb_clf.predict_proba(X_test)[:, 1]
lgbm_pred = lgb_clf.predict_proba(X_test)[:, 1]
cat_pred = cat_clf.predict_proba(X_test)[:, 1]
cat_pred_ = cat_clf_.predict_proba(X_test)[:, 1]
#rf_pred = rf_clf.predict_proba(X_test)[:,1]

In [ ]:
roc_auc = roc_auc_score(y_test, cat_pred)
print(f"ROC-AUC: {roc_auc:.6f}")

roc_auc = roc_auc_score(y_test, cat_pred_)
print(f"ROC-AUC: {roc_auc:.6f}")

roc_auc = roc_auc_score(y_test, lgbm_pred)
print(f"ROC-AUC: {roc_auc:.6f}")

roc_auc = roc_auc_score(y_test, xgb_pred)
print(f"ROC-AUC: {roc_auc:.6f}")

# roc_auc = roc_auc_score(y_test, rf_pred)
# print(f"ROC-AUC: {roc_auc:.6f}")

ROC-AUC: 0.739583
ROC-AUC: 0.739596
ROC-AUC: 0.739251
ROC-AUC: 0.739555


In [ ]:
blended_pred = (0.3 * xgb_pred + 0.3 * lgbm_pred + 0.4 * cat_pred_)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

blended_pred = (0.2 * xgb_pred + 0.4 * lgbm_pred + 0.4 * cat_pred_)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

blended_pred = (0.4 * xgb_pred + 0.2 * lgbm_pred + 0.4 * cat_pred_)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

blended_pred = (0.4 * xgb_pred + 0.3 * lgbm_pred + 0.3 * cat_pred_)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

blended_pred = (0.3 * xgb_pred + 0.4 * lgbm_pred + 0.3 * cat_pred_)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

blended_pred = (0.3 * xgb_pred + 0.2 * lgbm_pred + 0.5 * cat_pred_)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

blended_pred = (0.2 * xgb_pred + 0.3 * lgbm_pred + 0.5 * cat_pred_)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

blended_pred = (0.2 * xgb_pred + 0.35 * lgbm_pred + 0.45 * cat_pred_)  # 가중치 튜닝 가능 -> best
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

blended_pred = (0.5 * xgb_pred + 0.2 * lgbm_pred + 0.3 * cat_pred_)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

blended_pred = (0.2 * xgb_pred + 0.5 * lgbm_pred + 0.3 * cat_pred_)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

blended_pred = (0.3 * xgb_pred + 0.5 * lgbm_pred + 0.2 * cat_pred_)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

Blended ROC-AUC: 0.739819
Blended ROC-AUC: 0.739790
Blended ROC-AUC: 0.739827
Blended ROC-AUC: 0.739821
Blended ROC-AUC: 0.739790
Blended ROC-AUC: 0.739812
Blended ROC-AUC: 0.739808
Blended ROC-AUC: 0.739804
Blended ROC-AUC: 0.739825
Blended ROC-AUC: 0.739744
Blended ROC-AUC: 0.739731


In [ ]:
blended_pred = (0.3 * xgb_pred + 0.2 * lgbm_pred + 0.4 * cat_pred + 0.1 * rf_pred)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

blended_pred = (0.3 * xgb_pred + 0.3 * lgbm_pred + 0.3 * cat_pred + 0.1 * rf_pred)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

In [ ]:
blended_pred = (0.7 * xgb_pred + 0.3 * cat_pred)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

Blended ROC-AUC: 0.739615


In [ ]:
blended_pred = (0.3 * xgb_pred + 0.5 * lgbm_pred + 0.2 * cat_pred)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

Blended ROC-AUC: 0.739710


In [ ]:
X = df.drop(columns=['임신 성공 여부', '임신 시도 또는 마지막 임신 경과 연수'])
y = df['임신 성공 여부']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

cat_clf = CatBoostClassifier(**cat_params)
cat_clf.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=0)

lgb_clf = LGBMClassifier(**light_params, random_state = 42, verbose=-1)
lgb_clf.fit(X_train, y_train)

xgb_clf = XGBClassifier(**xg_params, use_label_encoder=False, eval_metric='auc', verbosity=0)
xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
xgb_pred = xgb_clf.predict_proba(X_test)[:, 1]
lgbm_pred = lgb_clf.predict_proba(X_test)[:, 1]
cat_pred = cat_clf.predict_proba(X_test)[:, 1]

In [ ]:
roc_auc = roc_auc_score(y_test, cat_pred)
print(f"ROC-AUC: {roc_auc:.6f}")

roc_auc = roc_auc_score(y_test, lgbm_pred)
print(f"ROC-AUC: {roc_auc:.6f}")

roc_auc = roc_auc_score(y_test, xgb_pred)
print(f"ROC-AUC: {roc_auc:.6f}")

ROC-AUC: 0.739677
ROC-AUC: 0.739343
ROC-AUC: 0.739571


In [ ]:
blended_pred = (0.3 * xgb_pred + 0.3 * lgbm_pred + 0.4 * cat_pred)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

blended_pred = (0.2 * xgb_pred + 0.4 * lgbm_pred + 0.4 * cat_pred)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

blended_pred = (0.4 * xgb_pred + 0.2 * lgbm_pred + 0.4 * cat_pred)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

blended_pred = (0.4 * xgb_pred + 0.3 * lgbm_pred + 0.3 * cat_pred)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

blended_pred = (0.3 * xgb_pred + 0.4 * lgbm_pred + 0.3 * cat_pred)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

blended_pred = (0.3 * xgb_pred + 0.2 * lgbm_pred + 0.5 * cat_pred)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

blended_pred = (0.2 * xgb_pred + 0.3 * lgbm_pred + 0.5 * cat_pred)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

blended_pred = (0.2 * xgb_pred + 0.35 * lgbm_pred + 0.45 * cat_pred)  # 가중치 튜닝 가능 -> best
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

blended_pred = (0.5 * xgb_pred + 0.2 * lgbm_pred + 0.3 * cat_pred)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

blended_pred = (0.2 * xgb_pred + 0.5 * lgbm_pred + 0.3 * cat_pred)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

blended_pred = (0.3 * xgb_pred + 0.5 * lgbm_pred + 0.2 * cat_pred)  # 가중치 튜닝 가능
roc_auc = roc_auc_score(y_test, blended_pred)
print(f"Blended ROC-AUC: {roc_auc:.6f}")

Blended ROC-AUC: 0.739879
Blended ROC-AUC: 0.739862
Blended ROC-AUC: 0.739876
Blended ROC-AUC: 0.739873
Blended ROC-AUC: 0.739857
Blended ROC-AUC: 0.739872
Blended ROC-AUC: 0.739873
Blended ROC-AUC: 0.739871
Blended ROC-AUC: 0.739871
Blended ROC-AUC: 0.739824
Blended ROC-AUC: 0.739799


In [ ]:
import numpy as np

test_df = pd.read_csv('/content/drive/MyDrive/LG Aimers 6기/Random Forest/Data/test_2.csv')
test_df = test_df.drop(columns = ['ID'])
X_test = test_df

cat_pred_ = cat_clf_.predict_proba(X_test)[:, 1]
lgbm_pred = lgb_clf.predict_proba(X_test)[:, 1]
xgb_pred = xgb_clf.predict_proba(X_test)[:, 1]

blended_pred = (0.4 * xgb_pred + 0.2 * lgbm_pred + 0.4 * cat_pred_)

num_samples = len(xgb_pred)
ids = [f"TEST_{i:05d}" for i in range(num_samples)]

results_df = pd.DataFrame({
    "ID": ids,
    "probability": blended_pred
})

results_df.to_csv('/content/drive/MyDrive/LG Aimers 6기/Random Forest/Prediction/prediction_0226_3.csv', index=False, encoding='utf-8-sig')
results_df.head()

,ID,probability
0,TEST_00000,0.001396
1,TEST_00001,0.002298
2,TEST_00002,0.181203
3,TEST_00003,0.123538
4,TEST_00004,0.527426


In [ ]:
results_df.head()

,ID,probability
0,TEST_00000,0.017696
1,TEST_00001,0.013782
2,TEST_00002,0.240109
3,TEST_00003,0.167522
4,TEST_00004,0.625221
